# 🔄 Análisis de Elasticidad Precio-Demanda - RGM Analytics

## Objetivo
Realizar un análisis completo de elasticidad precio-demanda para optimizar la estrategia de precios.

**Elasticidad de Precio**: Medida de la sensibilidad de la demanda ante cambios en el precio.
- **E < -1**: Producto elástico (demanda sensible al precio)
- **-1 < E < 0**: Producto inelástico (demanda poco sensible al precio)
- **E > 0**: Comportamiento atípico (posible bien de lujo)

## Metodología
1. Identificación de productos candidatos con suficiente variabilidad de precios
2. Segmentación inteligente de precios por producto
3. Cálculo de elasticidad usando regresión log-log
4. Validación estadística y categorización
5. Insights de negocio y recomendaciones

## 1. Configuración e Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import warnings
from scipy import stats

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📊 Librerías cargadas exitosamente")
print("🎯 Iniciando análisis de elasticidad precio-demanda...")

## 2. Funciones Auxiliares

In [ ]:
def clean_numeric_data(df, columns):
    """Limpia columnas numéricas con formato decimal español"""
    df_clean = df.copy()
    for col in columns:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].astype(str).str.replace(',', '.', regex=False)
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    return df_clean

def create_smart_price_segments(prices, method='dynamic'):
    """Crea segmentos de precios inteligentes según la distribución"""
    unique_prices = prices.nunique()
    
    if method == 'dynamic':
        if unique_prices >= 8:
            return pd.qcut(prices, q=5, labels=False, duplicates='drop')
        elif unique_prices >= 5:
            return pd.qcut(prices, q=4, labels=False, duplicates='drop')
        elif unique_prices >= 3:
            return pd.qcut(prices, q=3, labels=False, duplicates='drop')
        else:
            return None
    
    elif method == 'percentile':
        try:
            percentiles = [0, 25, 50, 75, 100]
            return pd.cut(prices, bins=np.percentile(prices, percentiles), 
                         labels=False, include_lowest=True, duplicates='drop')
        except:
            return None
    
    elif method == 'std_based':
        try:
            mean_price = prices.mean()
            std_price = prices.std()
            bins = [prices.min(), mean_price - std_price, mean_price, 
                   mean_price + std_price, prices.max()]
            bins = sorted(list(set([b for b in bins if b == b])))  # Remove NaN
            if len(bins) >= 3:
                return pd.cut(prices, bins=bins, labels=False, include_lowest=True)
            return None
        except:
            return None

def calculate_elasticity_robust(sku_data):
    """Calcula elasticidad usando el mejor método de segmentación"""
    methods = ['dynamic', 'percentile', 'std_based']
    best_result = None
    best_r2 = -1
    
    for method in methods:
        try:
            segments = create_smart_price_segments(sku_data['PRECIO'], method)
            if segments is None:
                continue
            
            sku_temp = sku_data.copy()
            sku_temp['Price_Segment'] = segments
            
            # Agregar por segmento
            agg_data = sku_temp.groupby('Price_Segment').agg({
                'PRECIO': 'mean',
                'QTY_ENTREGADA': 'sum',
                'IMPORTE': 'sum'
            }).reset_index()
            
            # Filtrar datos válidos
            agg_data = agg_data[
                (agg_data['PRECIO'] > 0) & 
                (agg_data['QTY_ENTREGADA'] > 0)
            ].dropna()
            
            if len(agg_data) < 3:
                continue
            
            # Calcular elasticidad con regresión log-log
            log_price = np.log(agg_data['PRECIO'])
            log_qty = np.log(agg_data['QTY_ENTREGADA'])
            
            if log_price.std() == 0 or log_qty.std() == 0:
                continue
            
            X = log_price.values.reshape(-1, 1)
            y = log_qty.values
            
            model = LinearRegression().fit(X, y)
            elasticity = model.coef_[0]
            r2 = model.score(X, y)
            
            if r2 > best_r2:
                best_r2 = r2
                best_result = {
                    'elasticity': elasticity,
                    'r2': r2,
                    'method': method,
                    'segments': len(agg_data),
                    'agg_data': agg_data,
                    'model': model
                }
        except Exception as e:
            continue
    
    return best_result

def categorize_elasticity(elasticity):
    """Categoriza la elasticidad en términos de negocio"""
    if elasticity > -0.5:
        return "Muy Inelástico", "🟢", "Alto potencial de aumento de precio"
    elif elasticity > -1.0:
        return "Inelástico", "🟡", "Moderado potencial de aumento de precio"
    elif elasticity > -2.0:
        return "Elástico", "🟠", "Cuidado con aumentos de precio"
    else:
        return "Muy Elástico", "🔴", "Evitar aumentos de precio"

print("✅ Funciones auxiliares definidas")

## 3. Carga y Preparación de Datos

In [ ]:
# Cargar datos
print("📂 Cargando datos...")
try:
    # Usar una muestra representativa para el análisis
    df_ventas = pd.read_csv('../data/processed/datos_limpios.csv', nrows=200000)
    print(f"✅ Datos cargados: {len(df_ventas):,} registros")
    print(f"📅 Periodo: {df_ventas['FECHA'].min()} a {df_ventas['FECHA'].max()}")
    
    # Mostrar información básica
    print(f"\n📊 Información básica:")
    print(f"   • Productos únicos: {df_ventas['SKU'].nunique():,}")
    print(f"   • Clientes únicos: {df_ventas['CLIENTE'].nunique():,}")
    print(f"   • Columnas: {list(df_ventas.columns)}")
    
except FileNotFoundError:
    print("❌ Error: No se encontró el archivo de datos")
    print("   Asegúrate de que exists 'data/processed/datos_limpios.csv'")
    raise

In [ ]:
# Limpiar datos numéricos
print("🧹 Limpiando formato numérico...")
numeric_cols = ['PRECIO', 'QTY_ENTREGADA', 'IMPORTE']
df_clean = clean_numeric_data(df_ventas, numeric_cols)

# Filtrar datos válidos
df_valid = df_clean[
    (df_clean['PRECIO'] > 0) & 
    (df_clean['QTY_ENTREGADA'] > 0) & 
    (df_clean['IMPORTE'] > 0) &
    df_clean['PRECIO'].notna() & 
    df_clean['QTY_ENTREGADA'].notna()
].copy()

print(f"✅ Registros válidos: {len(df_valid):,} de {len(df_ventas):,} ({len(df_valid)/len(df_ventas)*100:.1f}%)")

# Estadísticas descriptivas
print(f"\n📈 Estadísticas de precios:")
print(f"   • Precio promedio: ${df_valid['PRECIO'].mean():.2f}")
print(f"   • Precio mediano: ${df_valid['PRECIO'].median():.2f}")
print(f"   • Rango: ${df_valid['PRECIO'].min():.2f} - ${df_valid['PRECIO'].max():.2f}")
print(f"   • Desviación estándar: ${df_valid['PRECIO'].std():.2f}")

## 4. Identificación de Productos Candidatos

In [ ]:
print("🎯 Identificando productos candidatos para análisis de elasticidad...")

# Análisis de variabilidad por producto
product_summary = df_valid.groupby('SKU').agg({
    'PRECIO': ['count', 'mean', 'std', 'nunique', 'min', 'max'],
    'QTY_ENTREGADA': 'sum',
    'IMPORTE': 'sum'
}).reset_index()

# Aplanar nombres de columnas
product_summary.columns = ['SKU', 'transactions', 'price_mean', 'price_std', 
                          'price_unique', 'price_min', 'price_max', 
                          'qty_total', 'revenue_total']

# Calcular métricas de variabilidad
product_summary['price_cv'] = product_summary['price_std'] / product_summary['price_mean']
product_summary['price_range_pct'] = (product_summary['price_max'] - product_summary['price_min']) / product_summary['price_mean']
product_summary['price_density'] = product_summary['price_unique'] / product_summary['transactions']

print(f"\n📊 Análisis de {len(product_summary):,} productos únicos:")

# Criterios de selección (progresivamente más flexibles)
criteria_sets = [
    {
        'name': 'Estrictos',
        'min_transactions': 50,
        'min_cv': 0.20,
        'min_unique': 5,
        'min_range': 0.30
    },
    {
        'name': 'Moderados', 
        'min_transactions': 30,
        'min_cv': 0.15,
        'min_unique': 4,
        'min_range': 0.25
    },
    {
        'name': 'Flexibles',
        'min_transactions': 20,
        'min_cv': 0.10,
        'min_unique': 3,
        'min_range': 0.20
    },
    {
        'name': 'Muy Flexibles',
        'min_transactions': 15,
        'min_cv': 0.08,
        'min_unique': 3,
        'min_range': 0.15
    }
]

candidates = None
selected_criteria = None

for criteria in criteria_sets:
    temp_candidates = product_summary[
        (product_summary['transactions'] >= criteria['min_transactions']) &
        (product_summary['price_cv'] >= criteria['min_cv']) &
        (product_summary['price_unique'] >= criteria['min_unique']) &
        (product_summary['price_range_pct'] >= criteria['min_range'])
    ]
    
    print(f"   • Criterios {criteria['name']}: {len(temp_candidates)} candidatos")
    
    if len(temp_candidates) >= 5:  # Mínimo aceptable
        candidates = temp_candidates
        selected_criteria = criteria
        break

if candidates is None or len(candidates) == 0:
    print("\n⚠️ No se encontraron suficientes candidatos. Usando criterios mínimos...")
    candidates = product_summary[
        (product_summary['transactions'] >= 10) &
        (product_summary['price_cv'] >= 0.05) &
        (product_summary['price_unique'] >= 2)
    ]
    selected_criteria = {'name': 'Mínimos'}

print(f"\n✅ Productos candidatos seleccionados: {len(candidates)} (criterios {selected_criteria['name']})")

In [ ]:
# Visualizar distribución de candidatos
if len(candidates) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('📊 Distribución de Productos Candidatos para Análisis de Elasticidad', 
                 fontsize=16, fontweight='bold')
    
    # 1. Distribución de Coeficiente de Variación
    axes[0,0].hist(candidates['price_cv'], bins=20, alpha=0.7, color='steelblue', edgecolor='black')
    axes[0,0].axvline(candidates['price_cv'].mean(), color='red', linestyle='--', 
                     label=f'Media: {candidates["price_cv"].mean():.3f}')
    axes[0,0].set_title('Coeficiente de Variación de Precios')
    axes[0,0].set_xlabel('CV de Precios')
    axes[0,0].set_ylabel('Frecuencia')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. Transacciones vs CV
    scatter = axes[0,1].scatter(candidates['transactions'], candidates['price_cv'], 
                               alpha=0.6, c=candidates['revenue_total'], 
                               cmap='viridis', s=50, edgecolors='black', linewidth=0.5)
    axes[0,1].set_title('Transacciones vs Coeficiente de Variación')
    axes[0,1].set_xlabel('Número de Transacciones')
    axes[0,1].set_ylabel('CV de Precios')
    axes[0,1].set_xscale('log')
    plt.colorbar(scatter, ax=axes[0,1], label='Ingresos Totales')
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. Rango de Precios
    axes[1,0].hist(candidates['price_range_pct'] * 100, bins=20, alpha=0.7, 
                  color='orange', edgecolor='black')
    axes[1,0].axvline(candidates['price_range_pct'].mean() * 100, color='red', 
                     linestyle='--', label=f'Media: {candidates["price_range_pct"].mean()*100:.1f}%')
    axes[1,0].set_title('Rango de Variación de Precios')
    axes[1,0].set_xlabel('Rango de Precios (%)')
    axes[1,0].set_ylabel('Frecuencia')
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)
    
    # 4. Precios únicos vs Transacciones
    axes[1,1].scatter(candidates['transactions'], candidates['price_unique'], 
                     alpha=0.6, c=candidates['price_cv'], cmap='plasma', 
                     s=50, edgecolors='black', linewidth=0.5)
    axes[1,1].set_title('Diversidad de Precios vs Volumen')
    axes[1,1].set_xlabel('Número de Transacciones')
    axes[1,1].set_ylabel('Precios Únicos')
    axes[1,1].set_xscale('log')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Top candidatos
    print("\n🏆 Top 15 productos candidatos (por CV de precios):")
    top_candidates = candidates.nlargest(15, 'price_cv')
    print(f"{'SKU':<15} {'Trans':<8} {'CV':<8} {'Rango%':<8} {'Únicos':<8} {'Precio$':<10} {'Ingresos$':<12}")
    print("-" * 85)
    
    for _, row in top_candidates.iterrows():
        print(f"{row['SKU'][:14]:<15} {row['transactions']:<8.0f} {row['price_cv']:<8.3f} "
              f"{row['price_range_pct']*100:<8.1f} {row['price_unique']:<8.0f} "
              f"{row['price_mean']:<10.2f} {row['revenue_total']:<12.0f}")
else:
    print("⚠️ No hay candidatos suficientes para mostrar")

## 5. Cálculo de Elasticidades

In [ ]:
# Configuración del análisis
MAX_PRODUCTS_TO_ANALYZE = 25  # Limitar para performance y claridad
MIN_R2 = 0.15  # R² mínimo para considerar válido

print(f"🔄 Calculando elasticidades para los top {MAX_PRODUCTS_TO_ANALYZE} productos...")
print(f"📊 Criterio de validación: R² ≥ {MIN_R2}")

# Seleccionar productos para análisis
products_to_analyze = candidates.nlargest(MAX_PRODUCTS_TO_ANALYZE, 'price_cv')['SKU'].tolist()

# Calcular elasticidades
elasticity_results = []
analysis_details = []

for i, sku in enumerate(products_to_analyze):
    print(f"   Procesando {i+1}/{len(products_to_analyze)}: {sku}")
    
    # Extraer datos del producto
    sku_data = df_valid[df_valid['SKU'] == sku].copy()
    
    # Calcular elasticidad
    result = calculate_elasticity_robust(sku_data)
    
    if result is not None and result['r2'] >= MIN_R2:
        # Información del producto
        product_info = candidates[candidates['SKU'] == sku].iloc[0]
        
        elasticity_data = {
            'SKU': sku,
            'elasticity': result['elasticity'],
            'r2': result['r2'],
            'method': result['method'],
            'segments': result['segments'],
            'transactions': len(sku_data),
            'price_mean': product_info['price_mean'],
            'price_cv': product_info['price_cv'],
            'price_range_pct': product_info['price_range_pct'],
            'quantity_total': product_info['qty_total'],
            'revenue_total': product_info['revenue_total']
        }
        
        # Categorizar elasticidad
        category, emoji, recommendation = categorize_elasticity(result['elasticity'])
        elasticity_data['category'] = category
        elasticity_data['emoji'] = emoji
        elasticity_data['recommendation'] = recommendation
        
        elasticity_results.append(elasticity_data)
        
        # Guardar detalles para visualización posterior
        analysis_details.append({
            'SKU': sku,
            'result': result,
            'raw_data': sku_data
        })

print(f"\n✅ Análisis completado: {len(elasticity_results)} productos con elasticidad válida")

if len(elasticity_results) == 0:
    print("❌ No se encontraron productos con elasticidad estadísticamente significativa")
    print("   Recomendaciones:")
    print("   • Reducir el R² mínimo requerido")
    print("   • Usar un periodo de datos más amplio")
    print("   • Verificar la calidad de los datos de precios")
else:
    # Convertir a DataFrame para análisis
    results_df = pd.DataFrame(elasticity_results)
    
    # Filtrar elasticidades extremas
    valid_results = results_df[
        (results_df['elasticity'] > -10) &  # Eliminar valores muy extremos
        (results_df['elasticity'] < 5)     # Incluir algunos positivos por si son válidos
    ].copy()
    
    print(f"📈 Resultados válidos después de filtrar extremos: {len(valid_results)}")
    
    if len(valid_results) > 0:
        # Separar por signo de elasticidad
        negative_elasticity = valid_results[valid_results['elasticity'] < 0]
        positive_elasticity = valid_results[valid_results['elasticity'] > 0]
        
        print(f"   • Elasticidad negativa (comportamiento normal): {len(negative_elasticity)}")
        print(f"   • Elasticidad positiva (comportamiento atípico): {len(positive_elasticity)}")
        
        # Usar principalmente productos con elasticidad negativa para el análisis principal
        main_results = negative_elasticity if len(negative_elasticity) > 0 else valid_results

In [ ]:
# Mostrar resultados detallados
if len(elasticity_results) > 0 and 'main_results' in locals():
    print("\n" + "="*100)
    print("🎯 RESULTADOS DE ELASTICIDAD PRECIO-DEMANDA")
    print("="*100)
    
    # Ordenar por elasticidad (de más a menos elástico)
    display_results = main_results.sort_values('elasticity').reset_index(drop=True)
    
    print(f"\n{'#':<3} {'SKU':<15} {'Elasticidad':<12} {'R²':<8} {'Categoría':<15} "
          f"{'Método':<12} {'Precio$':<10} {'Recomendación':<30}")
    print("-" * 120)
    
    for idx, row in display_results.iterrows():
        print(f"{idx+1:<3} {row['SKU'][:14]:<15} {row['elasticity']:<12.3f} "
              f"{row['r2']:<8.3f} {row['emoji']} {row['category']:<13} "
              f"{row['method']:<12} ${row['price_mean']:<9.2f} {row['recommendation']:<30}")
    
    # Estadísticas resumidas
    print(f"\n📊 ESTADÍSTICAS RESUMIDAS:")
    print(f"   • Productos analizados: {len(display_results)}")
    print(f"   • Elasticidad promedio: {display_results['elasticity'].mean():.3f}")
    print(f"   • Elasticidad mediana: {display_results['elasticity'].median():.3f}")
    print(f"   • Rango: {display_results['elasticity'].min():.3f} a {display_results['elasticity'].max():.3f}")
    print(f"   • R² promedio: {display_results['r2'].mean():.3f}")
    print(f"   • Precio promedio: ${display_results['price_mean'].mean():.2f}")
    
    # Distribución por categorías
    category_dist = display_results['category'].value_counts()
    print(f"\n🏷️ DISTRIBUCIÓN POR CATEGORÍAS:")
    for category, count in category_dist.items():
        pct = count / len(display_results) * 100
        emoji = display_results[display_results['category'] == category]['emoji'].iloc[0]
        print(f"   {emoji} {category}: {count} productos ({pct:.1f}%)")
    
    # Insights de negocio
    print(f"\n💡 INSIGHTS DE NEGOCIO:")
    
    avg_elasticity = display_results['elasticity'].mean()
    if avg_elasticity > -1:
        print("   🟢 La mayoría de productos muestran demanda inelástica")
        print("   📈 Existe oportunidad para aumentos estratégicos de precio")
        print("   💰 Enfoque en maximización de margen por unidad")
    else:
        print("   🟠 Los productos muestran sensibilidad significativa al precio")
        print("   ⚠️ Evaluar cuidadosamente cualquier cambio de precio")
        print("   📊 Considerar estrategias de volumen vs. margen")
    
    # Productos específicos de interés
    most_inelastic = display_results.loc[display_results['elasticity'].idxmax()]
    most_elastic = display_results.loc[display_results['elasticity'].idxmin()]
    
    print(f"\n🎯 PRODUCTOS DESTACADOS:")
    print(f"   • Menos sensible al precio: {most_inelastic['SKU']} (E = {most_inelastic['elasticity']:.3f})")
    print(f"     → {most_inelastic['recommendation']}")
    print(f"   • Más sensible al precio: {most_elastic['SKU']} (E = {most_elastic['elasticity']:.3f})")
    print(f"     → {most_elastic['recommendation']}")
    
    # Productos con mejor ajuste estadístico
    best_fit = display_results.loc[display_results['r2'].idxmax()]
    print(f"   • Mejor modelo estadístico: {best_fit['SKU']} (R² = {best_fit['r2']:.3f})")
    
else:
    print("❌ No hay resultados válidos para mostrar")

## 6. Visualizaciones del Análisis

In [ ]:
# Dashboard principal de elasticidad
if len(elasticity_results) > 0 and 'display_results' in locals():
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('🎯 Dashboard de Análisis de Elasticidad Precio-Demanda', 
                 fontsize=18, fontweight='bold', y=0.98)
    
    # 1. Distribución de Elasticidades
    axes[0,0].hist(display_results['elasticity'], bins=15, alpha=0.7, 
                  color='steelblue', edgecolor='black', linewidth=1)
    axes[0,0].axvline(display_results['elasticity'].mean(), color='red', linestyle='--', 
                     linewidth=2, label=f'Media: {display_results["elasticity"].mean():.3f}')
    axes[0,0].axvline(-1, color='orange', linestyle='--', linewidth=2, 
                     label='Elasticidad Unitaria')
    axes[0,0].set_title('Distribución de Elasticidades', fontweight='bold')
    axes[0,0].set_xlabel('Elasticidad Precio-Demanda')
    axes[0,0].set_ylabel('Frecuencia')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. Calidad del Modelo vs Elasticidad
    scatter1 = axes[0,1].scatter(display_results['elasticity'], display_results['r2'], 
                                alpha=0.7, c=display_results['price_mean'], 
                                cmap='viridis', s=80, edgecolors='black', linewidth=1)
    axes[0,1].set_title('Calidad del Modelo vs Elasticidad', fontweight='bold')
    axes[0,1].set_xlabel('Elasticidad')
    axes[0,1].set_ylabel('R² (Calidad del Ajuste)')
    axes[0,1].grid(True, alpha=0.3)
    plt.colorbar(scatter1, ax=axes[0,1], label='Precio Promedio ($)')
    
    # 3. Categorización (Pie Chart)
    category_counts = display_results['category'].value_counts()
    colors = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99', '#ff99cc']
    wedges, texts, autotexts = axes[0,2].pie(category_counts.values, 
                                             labels=category_counts.index,
                                             autopct='%1.1f%%', startangle=90, 
                                             colors=colors[:len(category_counts)])
    axes[0,2].set_title('Categorización por Elasticidad', fontweight='bold')
    
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
    
    # 4. Elasticidad vs Precio Promedio
    scatter2 = axes[1,0].scatter(display_results['price_mean'], display_results['elasticity'], 
                                alpha=0.7, c=display_results['r2'], cmap='plasma', 
                                s=80, edgecolors='black', linewidth=1)
    axes[1,0].set_title('Elasticidad vs Precio Promedio', fontweight='bold')
    axes[1,0].set_xlabel('Precio Promedio ($)')
    axes[1,0].set_ylabel('Elasticidad')
    axes[1,0].grid(True, alpha=0.3)
    plt.colorbar(scatter2, ax=axes[1,0], label='R²')
    
    # 5. Volumen vs Elasticidad
    scatter3 = axes[1,1].scatter(display_results['quantity_total'], display_results['elasticity'], 
                                alpha=0.7, c=display_results['revenue_total'], 
                                cmap='coolwarm', s=80, edgecolors='black', linewidth=1)
    axes[1,1].set_title('Volumen vs Elasticidad', fontweight='bold')
    axes[1,1].set_xlabel('Cantidad Total Vendida')
    axes[1,1].set_ylabel('Elasticidad')
    axes[1,1].set_xscale('log')
    axes[1,1].grid(True, alpha=0.3)
    plt.colorbar(scatter3, ax=axes[1,1], label='Ingresos Totales ($)')
    
    # 6. Top Productos por Elasticidad (Horizontal Bar)
    top_n = min(12, len(display_results))
    top_products = display_results.nsmallest(top_n, 'elasticity')
    
    y_pos = np.arange(len(top_products))
    colors_bar = plt.cm.RdYlBu_r(np.linspace(0.2, 0.8, len(top_products)))
    
    bars = axes[1,2].barh(y_pos, top_products['elasticity'], color=colors_bar)
    axes[1,2].set_yticks(y_pos)
    axes[1,2].set_yticklabels([f'{sku[:10]}...' if len(sku) > 10 else sku 
                              for sku in top_products['SKU']], fontsize=9)
    axes[1,2].set_title(f'Top {top_n} Productos Más Elásticos', fontweight='bold')
    axes[1,2].set_xlabel('Elasticidad')
    axes[1,2].grid(True, alpha=0.3)
    
    # Añadir valores en las barras
    for i, (bar, val) in enumerate(zip(bars, top_products['elasticity'])):
        axes[1,2].text(val - 0.05, bar.get_y() + bar.get_height()/2, 
                      f'{val:.2f}', ha='right', va='center', 
                      fontweight='bold', fontsize=8, color='white')
    
    plt.tight_layout()
    plt.show()
    
else:
    print("📊 No hay datos suficientes para crear visualizaciones")

In [ ]:
# Gráfico interactivo con Plotly
if len(elasticity_results) > 0 and 'display_results' in locals():
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    
    # Crear subplots interactivos
    fig_interactive = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Elasticidad vs Precio', 'R² vs Elasticidad', 
                       'Volumen vs Elasticidad', 'Distribución de Categorías'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"type": "pie"}]]
    )
    
    # 1. Elasticidad vs Precio (con tamaño por volumen)
    fig_interactive.add_trace(
        go.Scatter(
            x=display_results['price_mean'],
            y=display_results['elasticity'],
            mode='markers',
            marker=dict(
                size=np.sqrt(display_results['quantity_total']) / 100,  # Normalizar tamaño
                color=display_results['r2'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="R²"),
                line=dict(width=1, color='black')
            ),
            text=display_results['SKU'],
            hovertemplate='<b>%{text}</b><br>' +
                         'Precio: $%{x:.2f}<br>' +
                         'Elasticidad: %{y:.3f}<br>' +
                         'R²: %{marker.color:.3f}<br>' +
                         '<extra></extra>',
            name='Productos'
        ),
        row=1, col=1
    )
    
    # 2. R² vs Elasticidad
    fig_interactive.add_trace(
        go.Scatter(
            x=display_results['elasticity'],
            y=display_results['r2'],
            mode='markers',
            marker=dict(
                size=10,
                color=display_results['category'].astype('category').cat.codes,
                colorscale='Set3',
                line=dict(width=1, color='black')
            ),
            text=display_results['SKU'],
            hovertemplate='<b>%{text}</b><br>' +
                         'Elasticidad: %{x:.3f}<br>' +
                         'R²: %{y:.3f}<br>' +
                         '<extra></extra>',
            showlegend=False
        ),
        row=1, col=2
    )
    
    # 3. Volumen vs Elasticidad
    fig_interactive.add_trace(
        go.Scatter(
            x=display_results['quantity_total'],
            y=display_results['elasticity'],
            mode='markers',
            marker=dict(
                size=display_results['price_mean'] / 5,  # Normalizar por precio
                color=display_results['revenue_total'],
                colorscale='RdYlBu',
                showscale=True,
                colorbar=dict(title="Ingresos ($)"),
                line=dict(width=1, color='black')
            ),
            text=display_results['SKU'],
            hovertemplate='<b>%{text}</b><br>' +
                         'Volumen: %{x:,.0f}<br>' +
                         'Elasticidad: %{y:.3f}<br>' +
                         '<extra></extra>',
            showlegend=False
        ),
        row=2, col=1
    )
    
    # 4. Distribución por categorías (Pie)
    category_counts = display_results['category'].value_counts()
    fig_interactive.add_trace(
        go.Pie(
            labels=category_counts.index,
            values=category_counts.values,
            hole=0.3,
            hovertemplate='<b>%{label}</b><br>' +
                         'Productos: %{value}<br>' +
                         'Porcentaje: %{percent}<br>' +
                         '<extra></extra>'
        ),
        row=2, col=2
    )
    
    # Configurar layout
    fig_interactive.update_xaxes(title_text="Precio Promedio ($)", row=1, col=1)
    fig_interactive.update_yaxes(title_text="Elasticidad", row=1, col=1)
    
    fig_interactive.update_xaxes(title_text="Elasticidad", row=1, col=2)
    fig_interactive.update_yaxes(title_text="R² (Calidad)", row=1, col=2)
    
    fig_interactive.update_xaxes(title_text="Volumen Total", type="log", row=2, col=1)
    fig_interactive.update_yaxes(title_text="Elasticidad", row=2, col=1)
    
    fig_interactive.update_layout(
        title=dict(
            text="🎯 Dashboard Interactivo: Análisis de Elasticidad Precio-Demanda",
            x=0.5,
            font=dict(size=16)
        ),
        height=800,
        showlegend=False
    )
    
    fig_interactive.show()

else:
    print("📊 No hay datos para crear visualización interactiva")

## 7. Análisis Detallado de Casos Específicos

In [ ]:
# Mostrar análisis detallado de los casos más interesantes
if len(elasticity_results) > 0 and 'display_results' in locals() and len(analysis_details) > 0:
    print("\n" + "="*80)
    print("🔍 ANÁLISIS DETALLADO DE CASOS ESPECÍFICOS")
    print("="*80)
    
    # Seleccionar casos de interés
    cases_to_analyze = [
        ('Más Inelástico', display_results.loc[display_results['elasticity'].idxmax()]['SKU']),
        ('Más Elástico', display_results.loc[display_results['elasticity'].idxmin()]['SKU']),
        ('Mejor Ajuste', display_results.loc[display_results['r2'].idxmax()]['SKU'])
    ]
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    fig.suptitle('🔍 Análisis Detallado: Relación Precio-Demanda por Producto', 
                 fontsize=16, fontweight='bold')
    
    for idx, (case_name, sku) in enumerate(cases_to_analyze):
        # Encontrar detalles del análisis
        detail = next((d for d in analysis_details if d['SKU'] == sku), None)
        if detail is None:
            continue
            
        result = detail['result']
        agg_data = result['agg_data']
        
        # Información del producto
        product_info = display_results[display_results['SKU'] == sku].iloc[0]
        
        # Crear gráfico de dispersión precio-cantidad
        axes[idx].scatter(agg_data['Precio_Medio'], agg_data['Cantidad_Total'],
                         s=100, alpha=0.7, color=f'C{idx}', edgecolors='black', linewidth=1)
        
        # Línea de tendencia
        x_trend = np.linspace(agg_data['Precio_Medio'].min(), agg_data['Precio_Medio'].max(), 100)
        # Usar el modelo logarítmico para la tendencia
        log_x = np.log(x_trend)
        log_y_pred = result['model'].predict(log_x.reshape(-1, 1))
        y_trend = np.exp(log_y_pred)
        
        axes[idx].plot(x_trend, y_trend, '--', color='red', linewidth=2, alpha=0.8)
        
        # Configurar gráfico
        axes[idx].set_title(f'{case_name}\n{sku[:15]}', fontweight='bold')
        axes[idx].set_xlabel('Precio Promedio ($)')
        axes[idx].set_ylabel('Cantidad Vendida')
        axes[idx].grid(True, alpha=0.3)
        
        # Añadir información en el gráfico
        info_text = f"E = {product_info['elasticity']:.3f}\nR² = {product_info['r2']:.3f}\nMétodo: {product_info['method']}"
        axes[idx].text(0.05, 0.95, info_text, transform=axes[idx].transAxes, 
                      verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        # Añadir puntos con etiquetas
        for _, point in agg_data.iterrows():
            axes[idx].annotate(f'${point["Precio_Medio"]:.1f}', 
                              (point['Precio_Medio'], point['Cantidad_Total']),
                              xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    # Mostrar tabla de datos agregados para cada caso
    for case_name, sku in cases_to_analyze:
        detail = next((d for d in analysis_details if d['SKU'] == sku), None)
        if detail is None:
            continue
            
        product_info = display_results[display_results['SKU'] == sku].iloc[0]
        agg_data = detail['result']['agg_data']
        
        print(f"\n📋 {case_name}: {sku}")
        print(f"   Elasticidad: {product_info['elasticity']:.3f} ({product_info['category']})")
        print(f"   R²: {product_info['r2']:.3f} | Método: {product_info['method']} | Segmentos: {product_info['segments']}")
        print(f"   Precio promedio: ${product_info['price_mean']:.2f} | CV: {product_info['price_cv']:.3f}")
        print(f"   Recomendación: {product_info['recommendation']}")
        
        print(f"\n   Datos agregados por segmento de precio:")
        print(f"   {'Segmento':<10} {'Precio$':<10} {'Cantidad':<12} {'Ingresos$':<12}")
        print("   " + "-" * 50)
        for _, row in agg_data.iterrows():
            print(f"   {int(row['Price_Segment']):<10} {row['Precio_Medio']:<10.2f} "
                  f"{row['Cantidad_Total']:<12.0f} {row['Importe_Total']:<12.0f}")

else:
    print("🔍 No hay datos detallados para mostrar")

## 8. Simulación de Impacto de Cambios de Precio

In [ ]:
# Simulación de impacto de cambios de precio
if len(elasticity_results) > 0 and 'display_results' in locals():
    print("\n" + "="*80)
    print("🎯 SIMULACIÓN DE IMPACTO DE CAMBIOS DE PRECIO")
    print("="*80)
    
    # Seleccionar algunos productos representativos para simulación
    simulation_products = display_results.head(5).copy()
    price_changes = [-10, -5, 0, 5, 10, 15]  # Cambios porcentuales de precio
    
    print(f"\nSimulando cambios de precio para {len(simulation_products)} productos...")
    print(f"Cambios de precio a evaluar: {price_changes}%\n")
    
    # Crear DataFrame para resultados de simulación
    simulation_results = []
    
    for _, product in simulation_products.iterrows():
        sku = product['SKU']
        base_price = product['price_mean']
        base_quantity = product['quantity_total']
        elasticity = product['elasticity']
        
        print(f"📊 {sku} (E = {elasticity:.3f})")
        print(f"   Precio base: ${base_price:.2f} | Cantidad base: {base_quantity:,.0f}")
        print(f"   {'Δ Precio%':<10} {'Nuevo $':<10} {'Δ Cantidad%':<12} {'Nueva Cant':<12} {'Δ Ingresos%':<12}")
        print("   " + "-" * 70)
        
        for price_change in price_changes:
            # Calcular nuevo precio
            new_price = base_price * (1 + price_change / 100)
            
            # Calcular cambio en cantidad usando elasticidad
            # % cambio en cantidad = elasticidad × % cambio en precio
            quantity_change_pct = elasticity * price_change
            new_quantity = base_quantity * (1 + quantity_change_pct / 100)
            
            # Calcular ingresos
            base_revenue = base_price * base_quantity
            new_revenue = new_price * new_quantity
            revenue_change_pct = ((new_revenue - base_revenue) / base_revenue) * 100
            
            print(f"   {price_change:>8}% ${new_price:>9.2f} {quantity_change_pct:>11.1f}% "
                  f"{new_quantity:>11.0f} {revenue_change_pct:>11.1f}%")
            
            # Guardar para gráfico
            simulation_results.append({
                'SKU': sku,
                'price_change_pct': price_change,
                'new_price': new_price,
                'quantity_change_pct': quantity_change_pct,
                'revenue_change_pct': revenue_change_pct,
                'elasticity': elasticity
            })
        
        print()
    
    # Visualizar simulación
    sim_df = pd.DataFrame(simulation_results)
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle('🎯 Simulación: Impacto de Cambios de Precio en Ingresos', 
                 fontsize=16, fontweight='bold')
    
    # Gráfico 1: Cambio en ingresos vs cambio en precio
    for sku in simulation_products['SKU']:
        sku_data = sim_df[sim_df['SKU'] == sku]
        elasticity = sku_data['elasticity'].iloc[0]
        axes[0].plot(sku_data['price_change_pct'], sku_data['revenue_change_pct'], 
                    'o-', label=f'{sku[:12]} (E={elasticity:.2f})', linewidth=2, markersize=6)
    
    axes[0].axhline(y=0, color='black', linestyle='--', alpha=0.5)
    axes[0].axvline(x=0, color='black', linestyle='--', alpha=0.5)
    axes[0].set_title('Cambio en Ingresos vs Cambio en Precio')
    axes[0].set_xlabel('Cambio en Precio (%)')
    axes[0].set_ylabel('Cambio en Ingresos (%)')
    axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[0].grid(True, alpha=0.3)
    
    # Gráfico 2: Mapa de calor de impacto en ingresos
    pivot_data = sim_df.pivot(index='SKU', columns='price_change_pct', values='revenue_change_pct')
    im = axes[1].imshow(pivot_data.values, cmap='RdYlGn', aspect='auto')
    
    # Configurar etiquetas
    axes[1].set_xticks(range(len(price_changes)))
    axes[1].set_xticklabels([f'{x}%' for x in price_changes])
    axes[1].set_yticks(range(len(simulation_products)))
    axes[1].set_yticklabels([sku[:12] for sku in simulation_products['SKU']])
    
    # Añadir valores en el mapa
    for i in range(len(simulation_products)):
        for j in range(len(price_changes)):
            text = axes[1].text(j, i, f'{pivot_data.iloc[i, j]:.1f}%',
                               ha="center", va="center", color="black", fontweight='bold')
    
    axes[1].set_title('Mapa de Calor: Impacto en Ingresos')
    axes[1].set_xlabel('Cambio en Precio')
    axes[1].set_ylabel('Productos')
    
    plt.colorbar(im, ax=axes[1], label='Cambio en Ingresos (%)')
    plt.tight_layout()
    plt.show()
    
    # Insights de la simulación
    print("💡 INSIGHTS DE LA SIMULACIÓN:")
    
    # Productos con mejor potencial de aumento de precio
    best_price_increase = sim_df[
        (sim_df['price_change_pct'] == 10) & 
        (sim_df['revenue_change_pct'] > 0)
    ].nlargest(3, 'revenue_change_pct')
    
    if len(best_price_increase) > 0:
        print("\n🟢 Productos con mejor potencial para aumentos de precio (+10%):")
        for _, row in best_price_increase.iterrows():
            print(f"   • {row['SKU']}: +{row['revenue_change_pct']:.1f}% ingresos (E={row['elasticity']:.3f})")
    
    # Productos riesgosos para aumentos de precio
    risky_products = sim_df[
        (sim_df['price_change_pct'] == 10) & 
        (sim_df['revenue_change_pct'] < -5)
    ]
    
    if len(risky_products) > 0:
        print("\n🔴 Productos riesgosos para aumentos de precio (+10%):")
        for _, row in risky_products.iterrows():
            print(f"   • {row['SKU']}: {row['revenue_change_pct']:.1f}% ingresos (E={row['elasticity']:.3f})")
    
    print(f"\n📈 Recomendación general: Los productos con elasticidad > -1 son buenos candidatos para aumentos moderados de precio")

else:
    print("🎯 No hay datos para simulación de precios")

## 9. Exportar Resultados y Recomendaciones

In [ ]:
# Exportar resultados
if len(elasticity_results) > 0 and 'display_results' in locals():
    print("💾 Guardando resultados...")
    
    # 1. Resultados principales
    output_file = '../data/processed/resultados_elasticidad.csv'
    display_results.to_csv(output_file, index=False)
    print(f"✅ Resultados principales guardados en: {output_file}")
    
    # 2. Simulación de precios
    if 'sim_df' in locals():
        simulation_file = '../data/processed/simulacion_precios.csv'
        sim_df.to_csv(simulation_file, index=False)
        print(f"✅ Simulación de precios guardada en: {simulation_file}")
    
    # 3. Resumen ejecutivo
    executive_summary = {
        'fecha_analisis': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
        'productos_analizados': len(display_results),
        'elasticidad_promedio': float(display_results['elasticity'].mean()),
        'elasticidad_mediana': float(display_results['elasticity'].median()),
        'r2_promedio': float(display_results['r2'].mean()),
        'productos_inelasticos': int((display_results['elasticity'] > -1).sum()),
        'productos_elasticos': int((display_results['elasticity'] <= -1).sum()),
        'producto_mas_inelastico': display_results.loc[display_results['elasticity'].idxmax(), 'SKU'],
        'producto_mas_elastico': display_results.loc[display_results['elasticity'].idxmin(), 'SKU'],
        'precio_promedio': float(display_results['price_mean'].mean()),
        'cv_promedio': float(display_results['price_cv'].mean())
    }
    
    summary_file = '../data/processed/resumen_ejecutivo_elasticidad.json'
    import json
    with open(summary_file, 'w') as f:
        json.dump(executive_summary, f, indent=2)
    print(f"✅ Resumen ejecutivo guardado en: {summary_file}")
    
    # 4. Recomendaciones por categoría
    recommendations = {
        'Muy Inelástico': {
            'accion': 'Aumentar precios moderadamente (5-15%)',
            'justificacion': 'Baja sensibilidad al precio permite optimizar márgenes',
            'precaucion': 'Monitorear competencia y satisfacción del cliente'
        },
        'Inelástico': {
            'accion': 'Aumentar precios conservadoramente (3-8%)',
            'justificacion': 'Moderada sensibilidad permite mejoras de margen',
            'precaucion': 'Evaluar impacto en volúmenes importantes'
        },
        'Elástico': {
            'accion': 'Mantener o reducir precios ligeramente',
            'justificacion': 'Alta sensibilidad requiere estrategia de volumen',
            'precaucion': 'Considerar promociones y descuentos estratégicos'
        },
        'Muy Elástico': {
            'accion': 'Evitar aumentos, considerar reducciones',
            'justificacion': 'Extrema sensibilidad al precio',
            'precaucion': 'Buscar diferenciación no basada en precio'
        }
    }
    
    recommendations_file = '../data/processed/recomendaciones_pricing.json'
    with open(recommendations_file, 'w') as f:
        json.dump(recommendations, f, indent=2, ensure_ascii=False)
    print(f"✅ Recomendaciones guardadas en: {recommendations_file}")
    
    print(f"\n📋 RESUMEN DE ARCHIVOS GENERADOS:")
    print(f"   1. {output_file} - Resultados detallados de elasticidad")
    if 'sim_df' in locals():
        print(f"   2. {simulation_file} - Simulaciones de cambios de precio")
    print(f"   3. {summary_file} - Métricas clave para dashboard")
    print(f"   4. {recommendations_file} - Recomendaciones por categoría")

else:
    print("💾 No hay resultados para exportar")

## 10. Conclusiones y Próximos Pasos

In [ ]:
# Conclusiones finales
print("\n" + "="*100)
print("🎯 CONCLUSIONES Y PRÓXIMOS PASOS - ANÁLISIS DE ELASTICIDAD")
print("="*100)

if len(elasticity_results) > 0 and 'display_results' in locals():
    
    print(f"\n📊 RESULTADOS PRINCIPALES:")
    print(f"   • Productos analizados con elasticidad válida: {len(display_results)}")
    print(f"   • Elasticidad promedio: {display_results['elasticity'].mean():.3f}")
    print(f"   • Calidad promedio de modelos (R²): {display_results['r2'].mean():.3f}")
    
    # Distribución por categorías
    category_summary = display_results['category'].value_counts()
    print(f"\n🏷️ DISTRIBUCIÓN POR SENSIBILIDAD AL PRECIO:")
    for category, count in category_summary.items():
        pct = count / len(display_results) * 100
        emoji = display_results[display_results['category'] == category]['emoji'].iloc[0]
        print(f"   {emoji} {category}: {count} productos ({pct:.1f}%)")
    
    print(f"\n💡 INSIGHTS ESTRATÉGICOS:")
    
    inelastic_count = (display_results['elasticity'] > -1).sum()
    inelastic_pct = inelastic_count / len(display_results) * 100
    
    if inelastic_pct > 50:
        print(f"   🟢 OPORTUNIDAD: {inelastic_pct:.0f}% de productos son relativamente inelásticos")
        print(f"   📈 Potencial de optimización de precios en la mayoría del portafolio")
        print(f"   💰 Enfoque recomendado: Maximizar márgenes en productos inelásticos")
    else:
        print(f"   🟠 PRECAUCIÓN: {100-inelastic_pct:.0f}% de productos son sensibles al precio")
        print(f"   📊 Requerirá estrategia mixta de precios y volumen")
        print(f"   🎯 Enfoque recomendado: Segmentación cuidadosa de estrategias")
    
    # Impacto económico estimado
    total_revenue = display_results['revenue_total'].sum()
    inelastic_revenue = display_results[display_results['elasticity'] > -1]['revenue_total'].sum()
    inelastic_revenue_pct = inelastic_revenue / total_revenue * 100
    
    print(f"\n💸 IMPACTO ECONÓMICO POTENCIAL:")
    print(f"   • Ingresos de productos inelásticos: ${inelastic_revenue:,.0f} ({inelastic_revenue_pct:.1f}%)")
    print(f"   • Potencial de aumento (5% precio en inelásticos): ${inelastic_revenue * 0.04:,.0f}")
    
else:
    print(f"\n⚠️ ANÁLISIS INCONCLUSO:")
    print(f"   • No se pudieron calcular elasticidades estadísticamente significativas")
    print(f"   • Posibles causas: datos insuficientes, poca variabilidad de precios")

print(f"\n🚀 PRÓXIMOS PASOS RECOMENDADOS:")
print(f"   1. 🎯 Implementar pruebas piloto de precio en productos inelásticos")
print(f"   2. 📊 Ampliar análisis con datos de más periodos temporales")
print(f"   3. 🔄 Integrar factores externos (competencia, estacionalidad)")
print(f"   4. 📈 Desarrollar modelo de optimización de precios dinámico")
print(f"   5. 🎭 Análizar elasticidades por segmento de cliente")
print(f"   6. ⚡ Automatizar monitoreo continuo de elasticidades")

print(f"\n📋 HERRAMIENTAS GENERADAS:")
print(f"   • Dashboard interactivo para exploración")
print(f"   • Simulador de impacto de cambios de precio")
print(f"   • Base de datos de elasticidades por producto")
print(f"   • Recomendaciones específicas por categoría")

print(f"\n" + "="*100)
print(f"✅ ANÁLISIS DE ELASTICIDAD COMPLETADO EXITOSAMENTE")
print(f"📅 Fecha: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔄 Recomendación: Revisar y actualizar cada 3-6 meses")
print(f"="*100)